## Поднимаем ElasticSearch

### Настраиваем окружение и поднимаем docker-контейнер

```bash
mkdir elasticsearch-docker
```

```bash
cd elasticsearch-docker
```


#### Собираем `docker-compose.yaml` файлик:

```yaml
version: '3.8'
services:
  elasticsearch:
    image: elasticsearch:8.17.2
    container_name: elasticsearch
    environment:
      - discovery.type=single-node
      - ES_JAVA_OPTS=-Xms512m -Xmx512m
      - xpack.security.enabled=false
    ports:
      - 9200:9200
    volumes:
      - es_data:/usr/share/elasticsearch/data
    networks:
      - es_network

  kibana:
    image: kibana:8.17.2
    container_name: kibana
    ports:
      - 5601:5601
    environment:
      - ELASTICSEARCH_HOSTS=http://elasticsearch:9200
    depends_on:
      - elasticsearch
    networks:
      - es_network

volumes:
  es_data:
    driver: local

networks:
  es_network:
    driver: bridge

```


#### Поднимаем контейнер

```bash
docker-compose up
```

### Проверяем, что всё поднялось и работает:

ElasticSearch: http://localhost:9200

Kibana: http://localhost:5601

### Подключение к ElasticSearch

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es = Elasticsearch("http://localhost:9200")

print(es.ping())  # Should return True

True


### Создание индекса

In [3]:
index_name = "my_first_index"

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

Index 'my_first_index' created.


### Индексация документов

https://www.elastic.co/search-labs/tutorials/search-tutorial/full-text-search/create-index

In [4]:
documents = [
    {"id": 1, "content": "Преступление и наказание"},
    {"id": 2, "content": "Мастер и Маргарита"},
    {"id": 3, "content": "Война и мир"},
]

for doc in documents:
    es.index(index=index_name, id=doc["id"], body=doc)
print("Documents indexed successfully.")

Documents indexed successfully.


### Поиск документов

In [5]:
query = {
    "query": {
        "match": {
            "content": "маргарита"
        }
    }
}

response = es.search(index=index_name, body=query)
print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'id': 2, 'content': 'Мастер и Маргарита'}


### Обновление и удаление документов

In [6]:
# Update a document
update_body = {
    "doc": {
        "content": "Наказание и преступление"
    }
}
es.update(index=index_name, id=1, body=update_body)
print("Document updated.")

# Delete a document
es.delete(index=index_name, id=3)
print("Document deleted.")

Document updated.
Document deleted.


### Удаление индекса

In [7]:
es.indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})

## Вернемся к нашим данным

In [8]:
import numpy as np
import pandas as pd

product_data = pd.read_parquet("products_with_names.parquet")

In [9]:
product_data.head()

product_id                                               name
0     4036767      Модуль сменный фильтрующий Аквафор КН, 208731
1     4050873  Водоочиститель Аквафор модель Кристалл Н, 2059...
2     4226160      Развиваем мышление (2-3 года) | Земцова Ольга
3     4644911          Lacoste Вода парфюмерная Pour Femme 50 мл
4     4788809  Сменные Кассеты Для Мужской Бритвы Gillette Ma...

In [10]:
from dataclasses import dataclass

@dataclass
class Document:
    doc_id: int
    name: str

documents = [Document(doc_id=doc[1]["product_id"], name=doc[1]["name"]) for doc in product_data.iterrows()]


In [11]:
index_name = "products"

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

Index 'products' already exists.


In [12]:
actions = [
    {"_index": "products", "_id": doc.doc_id, "_source": {"title": doc.name}}
    for doc in documents
]

bulk(es, actions)


(238443, [])

In [13]:
query = {
    "query": {
        "match": {
            "title": "молоко пастеризованное"
        }
    }
}

response = es.search(index=index_name, body=query)
print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'title': 'Молоко 2,5%, 950 мл, Калининское, пастеризованное'}
{'title': 'Молоко 3,2%, 930 мл, Экомилк, пастеризованное'}
{'title': 'Молоко пастеризованное Кубанская буренка 2.5% 1,4л'}
{'title': 'Молоко пастеризованное ЭкоНива, 2,5%, 1 л'}
{'title': 'Молоко пастеризованное Кубанская буренка Отборное 930мл'}
{'title': 'Молоко пастеризованное 2,5% 930 мл Простоквашино'}
{'title': 'Молоко 3,2%, 1400 мл, Простоквашино, пастеризованное'}
{'title': 'Молоко пастеризованное Кубанская буренка 2.5% 930мл'}
{'title': 'Молоко пастеризованное 3,2 % 1 кг, Вологжанка'}
{'title': 'Молоко пастеризованное 3,2% 1400 мл, Простоквашино'}


In [14]:
query = {
    "query": {
        "match": {
            "title": "сыр пармезан"
        }
    }
}

response = es.search(index=index_name, body=query)
print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'title': 'Сыр Пармезан 45% 180 г, Schonfeld'}
{'title': 'Сыр Пармезан 45%, 125 г, Schonfeld'}
{'title': 'Крем-сыр Cheese Pleasure Пармезан, 100 г'}
{'title': 'Сыр Пармезан 40% 80 г, Schonfeld, гранулы'}
{'title': 'Сыр твердый Ricrem Пармезан, 42%, 200 г'}
{'title': 'Cheese Gallery Сыр Пармезан, 32%, хлопья, 100 г'}
{'title': 'Сыр Пармезан Dolce Platinum, 40 %, 160 - 180 г'}
{'title': 'Сыр твердый PALERMO пармезан, слайсы, 40 %, 120 г'}
{'title': 'Сыр Пармезан Dolce Platinum, 40 %, 120 - 139 г'}
{'title': 'Сыр твердый Пармезан GRANA 43% 190 г, Schonfeld'}


### Продвинутые настройки

https://www.elastic.co/guide/en/elasticsearch/reference/current/specify-analyzer.html

https://www.elastic.co/guide/en/elasticsearch/reference/current/analyzer.html

https://opster.com/guides/elasticsearch/data-architecture/elasticsearch-text-analyzers/

https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-update-settings.html#update-settings-analysis


In [15]:
advanced_settings = {
  "settings": {
    "analysis": {
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "whitespace",
          "char_filter": ["replace_yo_filter"],
          "filter": [
            "lowercase",
            "russian_stop",
            "english_stop"
          ]
        }
      },
      "char_filter": {
        "replace_yo_filter": {
          "type": "mapping",
          "mappings": ["ё => е"]
        }
      },
      "filter": {
        "russian_stop": {
          "type": "stop",
          "stopwords": "_russian_"
        },
        "english_stop": {
          "type": "stop",
          "stopwords": "_english_"
        },
      }
    },
    "similarity": {
        "default": {
            "type": "BM25",
            "k1": 1.2,
            "b": 0.75
        }
    }
  },
  "mappings": {
    "properties": {
      "content": {
        "type": "text",
        "analyzer": "custom_analyzer",
        "search_analyzer": "custom_analyzer"
      }
    }
  }
}

In [16]:
index_name = "products_with_advanced_settings"

if es.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists. Deleting it.")
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=advanced_settings)
print(f"Index '{index_name}' created.")


Index 'products_with_advanced_settings' already exists. Deleting it.
Index 'products_with_advanced_settings' created.


In [17]:
analyzer_test_text = "Искусственная ёлка и большая 2 метра"
analyzer_test = es.indices.analyze(
    index=index_name,
    body={
        "analyzer": "custom_analyzer",
        "text": analyzer_test_text
    }
)

In [18]:
print(analyzer_test)

{'tokens': [{'token': 'искусственная', 'start_offset': 0, 'end_offset': 13, 'type': 'word', 'position': 0}, {'token': 'елка', 'start_offset': 14, 'end_offset': 18, 'type': 'word', 'position': 1}, {'token': 'большая', 'start_offset': 21, 'end_offset': 28, 'type': 'word', 'position': 3}, {'token': '2', 'start_offset': 29, 'end_offset': 30, 'type': 'word', 'position': 4}, {'token': 'метра', 'start_offset': 31, 'end_offset': 36, 'type': 'word', 'position': 5}]}


In [19]:
actions = [
    {"_index": "products_with_advanced_settings",  "_id": doc.doc_id, "_source": {"title": doc.name}}
    for doc in documents
]

bulk(es, actions)

(238443, [])

In [20]:
query = {
    "query": {
        "match": {
            "title": {
                "query": "вода 5л",
                "analyzer": "custom_analyzer"
            }
        }
    }
}

response = es.search(index=index_name, body=query)
print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'title': 'Дистиллированная вода OILRIGHT 5л'}
{'title': 'Дистиллированная вода Аляска 5л ПЭТ'}
{'title': 'Дистиллированная вода SPECTROL Аква 5л'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев, Х4'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев, Х4. Уцененный товар'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев, Х4. Уцененный товар'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев, Х4. Уцененный товар'}
{'title': 'Вода детская питьевая Агуша 5л, с 0 месяцев, Х4. Уцененный товар'}


In [21]:
query = {
    "query": {
        "match": {
            "title": {
                "query": "ёлка",
                "analyzer": "custom_analyzer"
            }
        }
    }
}

response = es.search(index=index_name, body=query)
print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'title': 'Искусственная новогодняя елка Царь Елка Фаворит, литая, 220 см'}
{'title': 'Искусственная новогодняя елка Царь Елка Фаворит, литая, 160 см'}
{'title': 'Elki Lux Елка искусственная Елка, Из ПВХ, 130 см'}
{'title': 'Искусственная новогодняя елка Царь Елка Оникс, литая, 80 см'}
{'title': 'Искусственная новогодняя елка Царь Елка Алиса, литая, 160 см'}
{'title': 'Искусственная новогодняя елка Царь Елка Фаворит, литая, 190 см'}
{'title': 'Искусственная новогодняя елка Царь Елка Финская премиум, литая, 120 см'}
{'title': 'Искусственная новогодняя елка Царь Елка СМАЙЛ, из ПВХ, 210 см'}
{'title': 'Искусственная новогодняя елка Царь Елка СМАЙЛ, из ПВХ, 180 см'}
{'title': 'Искусственная новогодняя елка Царь Елка Инфинити 3D, литая, 120 см'}


In [24]:
response["hits"]["hits"]

[{'_index': 'products_with_advanced_settings',
  '_id': '578552626',
  '_score': 11.691586,
  '_source': {'title': 'Искусственная новогодняя елка Царь Елка Фаворит, литая, 220 см'}},
 {'_index': 'products_with_advanced_settings',
  '_id': '578552370',
  '_score': 11.691586,
  '_source': {'title': 'Искусственная новогодняя елка Царь Елка Фаворит, литая, 160 см'}},
 {'_index': 'products_with_advanced_settings',
  '_id': '742064005',
  '_score': 11.691586,
  '_source': {'title': 'Elki Lux Елка искусственная Елка, Из ПВХ, 130 см'}},
 {'_index': 'products_with_advanced_settings',
  '_id': '252851255',
  '_score': 11.691586,
  '_source': {'title': 'Искусственная новогодняя елка Царь Елка Оникс, литая, 80 см'}},
 {'_index': 'products_with_advanced_settings',
  '_id': '578553639',
  '_score': 11.691586,
  '_source': {'title': 'Искусственная новогодняя елка Царь Елка Алиса, литая, 160 см'}},
 {'_index': 'products_with_advanced_settings',
  '_id': '578553451',
  '_score': 11.691586,
  '_source':

In [25]:
def search_with_es(query, limit = 20):
    search_query = {
        "query": {
            "match": {
                "title": {
                    "query": query,  # Search term
                    "analyzer": "custom_analyzer"  # Explicitly specify the analyzer
                }
            }
        },
        "size": limit
    }
    
    response = es.search(index=index_name, body=search_query)
    return [int(x["_id"]) for x in response["hits"]["hits"]]

In [26]:
validation_query_positives = pd.read_parquet("validation_query_positives.parquet")

In [27]:
validation_query_positives_dict = {
    row[1].query: set(row[1].products.tolist()) for row in validation_query_positives.iterrows()
}

In [28]:
@dataclass
class Metrics:
    precision: float
    recall: float
    f1_score: float
        
    def __repr__(self):
        return f"precision = {self.precision}\nrecall = {self.recall}\nf1_score = {self.f1_score}"


In [29]:
def calculate_metrics(ground_truth_set, search_results_set):
    
    # True positives: items that are both in ground truth and search results
    tp = len(ground_truth_set.intersection(search_results_set))
    
    # Precision: tp / (tp + fp)
    precision = tp / len(search_results_set) if len(search_results_set) > 0 else 0.0
    
    # Recall: tp / (tp + fn)
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0.0
    
    # F1-score: harmonic mean of precision and recall
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return Metrics(precision=precision, recall=recall, f1_score=f1_score)


In [30]:
def calculate_validation_metrics(search_function, limit=20):
    metrics = []
    for query, positives in validation_query_positives_dict.items():
        metrics.append(
            calculate_metrics(positives, search_function(query=query, limit=limit))
        )
    
    return Metrics(
        precision=np.mean([x.precision for x in metrics]),
        recall=np.mean([x.recall for x in metrics]),
        f1_score=np.mean([x.f1_score for x in metrics]),
    )
    

In [31]:
calculate_validation_metrics(search_with_es, limit=200)

precision = 0.15846644458231876
recall = 0.21245135916618588
f1_score = 0.1527730349041491